In [79]:
import pandas as pd
import os
import json

In [2]:
df10 = pd.read_csv("Ref10_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df11 = pd.read_csv("Ref11_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df12 = pd.read_csv("Ref12_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df13 = pd.read_csv("Ref13_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )
df14 = pd.read_csv("Ref14_LMI.csv", index_col = 0, dtype = {'SHP_BOROUGH': object, 'SHP_CENSUS_TRACT': object, 'SHP_CENSUS_BLOCK': object, 'Geo_COUNTY': object} )

In [7]:
files = [df10, df11, df12, df13, df14]

for df in files:
    df['count'] = 1

In [53]:
df10_grouped = df10.groupby(['PRMSIC', 'PRMSICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df11_grouped = df11.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df12_grouped = df12.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df13_grouped = df13.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()
df14_grouped = df14.groupby(['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct'])['EMPSDT', 'SLSVDT', 'count'].agg(['sum', 'median']).reset_index()

In [54]:
grouped_files = [df10_grouped, df11_grouped, df12_grouped, df13_grouped, df14_grouped]

In [33]:
cols = ['PRMSIC', 'SICD', 'PNACODE', 'PNATITL', 'Geo_COUNTY', 'SHP_BOROUGH', 'SHP_CENSUS_TRACT', 'lmi_ct', \
        'EMPSDT_SUM', 'EMPSDT_MED', 'SLSVDT_SUM', 'SLSVDT_MED', 'count_SUM', 'count_MED']

In [28]:
def getNAICS(row):
    code = str(row['PNACODE'])[0:2]
    return code

In [52]:
def getNAICSLabel(row):
    label = naicsCODES[row['NAICS']] 
    return label

In [27]:
naicsCODES = \
{'11': 'AGRICULTURE, FORESTRY, FISHING AND HUNTING',
 '21': 'MINING',
 '22': 'UTILITIES',
 '23': 'CONSTRUCTION',
 '31': 'MANUFACTURING',
 '32': 'MANUFACTURING',
 '33': 'MANUFACTURING',
 '42': 'WHOLESALE TRADE',
 '44': 'RETAIL TRADE',
 '45': 'RETAIL TRADE',
 '48': 'TRANSPORTATION AND WAREHOUSING',
 '49': 'TRANSPORTATION AND WAREHOUSING',
 '51': 'INFORMATION',
 '52': 'FINANCE AND INSURANCE',
 '53': 'REAL ESTATE RENTAL AND LEASING',
 '54': 'PROFESSIONAL, SCIENTIFIC, AND TECHNICAL SERVICES',
 '55': 'MANAGEMENT OF COMPANIES AND ENTERPRISES',
 '56': 'ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMENT AND REMEDIATION SERVICES',
 '61': 'EDUCATIONAL SERVICES',
 '62': 'HEALTH CARE AND SOCIAL ASSISTANCE',
 '71': 'ARTS, ENTERTAINMENT, AND RECREATION',
 '72': 'ACCOMMODATION AND FOOD SERVICES',
 '81': 'OTHER SERVICES (EXCEPT PUBLIC ADMINISTRATION)',
 '92': 'PUBLIC ADMINISTRATION',
 '99': 'UNCLASSIFIED'}

In [55]:
for df in grouped_files:
    df.columns = df.columns.droplevel(1)
    df.columns = cols 
    del df['count_MED']
    df.loc[:,'NAICS'] = df.apply(getNAICS, axis = 1)
    df.loc[:,'NAICS_LABEL'] = df.apply(getNAICSLabel, axis = 1)

####don't use below code - no need to save

In [ ]:
df10_grouped.to_csv("df10_grouped.csv")
df11_grouped.to_csv("df11_grouped.csv")
df12_grouped.to_csv("df12_grouped.csv") 
df13_grouped.to_csv("df13_grouped.csv")
df14_grouped.to_csv("df14_grouped.csv") 

In [57]:
import geopandas as gp

In [58]:
boros = gp.GeoDataFrame.from_file("../nyct2010_15b/nyct2010.shp")

####test drive

In [107]:
df = df10_grouped[df10_grouped['SICD'] == 'BEAUTY SALONS']

In [109]:
result = pd.merge(df, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
del result['BoroName']
del result['CT2010']

####Create Directories with Files

In [100]:
df = grouped_files[4]
year = YEARS[4] 

base_year_loc = base_loc + "/" + year
year_dir = os.listdir(base_year_loc)

In [101]:
YEARS = ["2010", "2011", "2012", "2013", "2014"]
NAICS = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(0))
PNATITL = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(1))
SICD = list(df.groupby(['NAICS_LABEL', 'PNATITL', 'SICD']).agg('count').index.get_level_values(2))

base_loc = 
cnt = len(PNATITL)

In [102]:
def createGeoJSON(pnatitl_loc, string2):
    
    #pnatitl_loc = base_year_loc + "/" + string1
    sicd_code = string2.replace("/", "-") 
    
    df_temp = df[df['SICD'] == string2]
    
    result = pd.merge(df_temp, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])
    
    loc = pnatitl_loc + "/" + sicd_code + ".json"
    
    with open(loc, 'wb') as fp:
        json.dump(gp.GeoDataFrame(result).to_json(), fp)

#gp.GeoDataFrame(result).to_json()

In [103]:
for index in range(cnt): 
    
    year_dir = os.listdir(base_year_loc)
    
    naics = NAICS[index]
    pnatitl = PNATITL[index].replace("/", "-")
    sicd = SICD[index]
    
    if naics not in os.listdir(base_year_loc): 
        naics_loc = base_year_loc + "/" + naics
        os.mkdir(naics_loc)
        
        pnatitl_loc = naics_loc + "/" + pnatitl 
        os.mkdir(pnatitl_loc)
        #create file 
        createGeoJSON(pnatitl_loc, sicd)
        
    else: 
        naics_loc = base_year_loc + "/" + naics
        
        if pnatitl not in os.listdir(naics_loc):
            pnatitl_loc = naics_loc + "/" + pnatitl 
            os.mkdir(pnatitl_loc)
            #create file 
            createGeoJSON(pnatitl_loc, sicd)
            
        else: 
            
            pnatitl_loc = naics_loc + "/" + pnatitl 
            #create file 
            createGeoJSON(pnatitl_loc, sicd)     

In [70]:
for year in YEARS:
    year_folder = str(year)
    
    if year_folder in os.listdir(base_dir):
        print 'already created'
    else:
        add_year = base_dir + "/" + year_folder
        os.mkdir(add_year)

NameError: name 'base_dir' is not defined

In [92]:
apm = base_year_loc + "/" + "ABRASIVE PROD MANUFACTURING"
ac = base_year_loc + "/"  + "AC REFRIGERATION & FORCED AIR HEATING"

os.mkdir(apm)
os.mkdir(ac)

In [136]:
# df10_grouped.groupby(['PNATITL', 'SICD']).agg('count').head(10)

x = "INTERNET PUBLISHING/BROADCASTING/WEB SEARCH PRTLS"

In [138]:
x.replace("/", "-")

'INTERNET PUBLISHING-BROADCASTING-WEB SEARCH PRTLS'

In [172]:
result = df10_grouped[df10_grouped['PNATITL'] == 'DIRECT MAIL ADVERTISING']


In [174]:
result1 = pd.merge(result, boros[['BoroName', 'CT2010', 'NTAName', 'Shape_Area', 'geometry']], \
                  left_on = ['SHP_BOROUGH', 'SHP_CENSUS_TRACT'], right_on = ['BoroName', 'CT2010'])

In [182]:
#gp.GeoDataFrame(result1).to_json()
#result['SHP_CENSUS_TRACT'].value_counts()

In [ ]:
f